<a href="https://colab.research.google.com/github/Jino1907/Geo_tagged_tweets_visualization.github.io/blob/main/02_geosearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [4]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "UvdbOSmaKCwdwJGwTIiibYw3H"
    consumer_secret = "JIlekywuDhd7wa62smBYGo3XKL4gPzGksbaEvwP6W4SfO6DqmE"
    access_token = "1516462627999784962-t0IZSMXSaOVnaM2p59Sr7kIId8vdPc"
    access_token_secret = "JchAUAnLyMCiR78Y5jUBHfbc6QjkmTs2XeTIi1bfPQCZu"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    LOCATIONS1 = [-140.742656, 48.895519, -55.746095, 69.520160]
 

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations = LOCATIONS1, languages=['en'])

{'id': 1517607506385596417, 'username': 'JohannBerg', 'created_at': 'Fri Apr 22 20:53:19 +0000 2022', 'lng': -114.087851, 'lat': 51.027664, 'text': "@MarissaOchsner Guess you won't be walking on the sun"}
{'id': 1517607571435048965, 'username': 'safensoundfest', 'created_at': 'Fri Apr 22 20:53:35 +0000 2022', 'lng': -122.917508, 'lat': 49.2067515, 'text': 'Who’s tryna party with us? We’re coming in with some heavy hitters this year 🔥 Buckle up, because it’s going to get… https://t.co/N0nu4PtkaG'}
{'id': 1517607655316987904, 'username': 'mandy_rushton', 'created_at': 'Fri Apr 22 20:53:55 +0000 2022', 'lng': -123.13083649999999, 'lat': 49.1742415, 'text': 'April 22, 2022Happy Earth Day! Let your heart swell with gratitude for the place we call home.Warmly,The Good Trade Team'}
{'id': 1517607662292135942, 'username': 'slav_metalurges', 'created_at': 'Fri Apr 22 20:53:57 +0000 2022', 'lng': -113.4926175, 'lat': 53.5558195, 'text': 'This except Thirteen.'}
{'id': 1517607699625791488, 'usern

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
